In [ ]:
import pandas as pd

# 기상데이터_1 전처리

In [ ]:
df_1_actual = pd.read_csv('/content/drive/MyDrive/산업정보공학특론/사용데이터/통합기상실측데이터_1.csv')


<ipython-input-3-bb623323e0e4>:1: DtypeWarning: Columns (2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18) have mixed types. Specify dtype option on import or set low_memory=False.
  df_1_actual = pd.read_csv('/content/drive/MyDrive/산업정보공학특론/사용데이터/통합기상실측데이터_1.csv')


In [ ]:
df_1_predict = pd.read_csv('/content/drive/MyDrive/산업정보공학특론/사용데이터/통합기상예측데이터_1.csv')

In [ ]:
df_1_actual

,location,ts,temp,real_feel_temp,real_feel_temp_shade,rel_hum,dew_point,wind_dir,wind_spd,wind_gust_spd,uv_idx,vis,cld_cvr,ceiling,pressure,appr_temp,wind_chill_temp,wet_bulb_temp,precip_1h,new_ts
0,Ilgwa-ri,1.709222e+09,6.9,2.4,2.4,69.0,1.5,338.0,18.7,37.7,0.0,17.7,98.0,579.0,1022.0,8.3,3.3,4.5,0.0,2024-02-29 15:57:00
1,Ilgwa-ri,1.709223e+09,7.1,2.7,2.7,68.0,1.5,338.0,18.7,37.7,0.0,17.7,98.0,579.0,1022.0,8.9,3.9,4.6,0.0,2024-02-29 16:03:00
2,Ilgwa-ri,1.709226e+09,6.3,0.9,0.9,64.0,-0.1,338.0,22.0,22.0,0.0,16.1,91.0,579.0,1018.6,7.2,2.2,3.5,0.0,2024-02-29 17:02:00
3,Ilgwa-ri,1.709229e+09,5.6,-1.7,-1.7,63.0,-0.9,338.0,31.7,31.7,0.0,16.1,45.0,12192.0,1023.0,6.7,0.6,2.9,0.0,2024-02-29 17:57:00
4,Ilgwa-ri,1.709230e+09,5.6,-1.7,-1.7,63.0,-0.9,338.0,31.7,31.7,0.0,16.1,45.0,3018.0,1024.0,6.7,0.6,2.9,0.0,2024-02-29 18:04:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
68607,Sangmo-ri,1.730866e+09,16.3,14.1,12.3,60,8.4,23,36.8,36.8,3,22.5,12,9754,1023.4,18.3,16.1,11.9,0.0,2024-11-06 04:02:00
68608,Sangmo-ri,1.730869e+09,16.0,13.6,11.9,68,10.0,0,37.8,37.8,3,16.1,0,12192,1027.0,18.3,16.1,12.8,0.0,2024-11-06 04:56:00
68609,Sangmo-ri,1.730869e+09,15.8,12.9,11.5,69,10.0,0,37.8,37.8,3,16.1,0,10363,1023.4,18.3,15.6,12.7,0.0,2024-11-06 05:02:00
68610,Sangmo-ri,1.730873e+09,15.9,12.4,11.4,65,9.5,315,43.2,43.2,2,16.1,26,11278,1023.4,18.3,16.1,12.5,0.0,2024-11-06 06:03:00


## 실측_1 초반시간대만 살림

In [ ]:
df_1_actual['new_ts'] = pd.to_datetime(df_1_actual['new_ts'])

# 2. datetime에서 시간(hour) 정보만 추출
df_1_actual['hour'] = df_1_actual['new_ts'].dt.floor('H')  # 시간 단위로 내림 처리
df_1_actual['time'] = df_1_actual['new_ts'].dt.floor('H')
# 3. 동일 location, hour에서 가장 초반 데이터 선택
df_filtered = (
    df_1_actual.sort_values(by=['location', 'hour', 'new_ts'])
    .groupby(['location', 'hour'])
    .first()
    .reset_index()
)

# 4. 결과 데이터프레임
df_result_1_actual = df_filtered.drop(columns=['hour','new_ts','ts'])  # 필요 없어진 열 제거

<ipython-input-6-1b0d25f6ab57>:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_1_actual['hour'] = df_1_actual['new_ts'].dt.floor('H')  # 시간 단위로 내림 처리
<ipython-input-6-1b0d25f6ab57>:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_1_actual['time'] = df_1_actual['new_ts'].dt.floor('H')


In [ ]:
df_result_1_actual.set_index("time", inplace=True)

# 2. 열 이름을 `지역_변수` 형식으로 변경
# location별로 데이터 처리
df_list = []
for location in df_result_1_actual['location'].unique():
    # 특정 location의 데이터만 선택
    temp_df = df_result_1_actual[df_result_1_actual['location'] == location].copy()
    # 열 이름 변경
    temp_df.columns = [f"{location}_{col}_actual" if col != "location" else col for col in temp_df.columns]
    # location 열 삭제
    temp_df.drop(columns=["location"], inplace=True)
    # 결과를 리스트에 저장
    df_list.append(temp_df)

# 3. location별 데이터프레임 병합
df_renamed_1_actual = pd.concat(df_list, axis=1)

In [ ]:
df_renamed_1_actual

,Bonggae-dong_temp_actual,Bonggae-dong_real_feel_temp_actual,Bonggae-dong_real_feel_temp_shade_actual,Bonggae-dong_rel_hum_actual,Bonggae-dong_dew_point_actual,Bonggae-dong_wind_dir_actual,Bonggae-dong_wind_spd_actual,Bonggae-dong_wind_gust_spd_actual,Bonggae-dong_uv_idx_actual,Bonggae-dong_vis_actual,...,Yongsu-ri_wind_gust_spd_actual,Yongsu-ri_uv_idx_actual,Yongsu-ri_vis_actual,Yongsu-ri_cld_cvr_actual,Yongsu-ri_ceiling_actual,Yongsu-ri_pressure_actual,Yongsu-ri_appr_temp_actual,Yongsu-ri_wind_chill_temp_actual,Yongsu-ri_wet_bulb_temp_actual,Yongsu-ri_precip_1h_actual
time,,,,,,,,,,,,,,,,,,,,,
2024-02-29 15:00:00,7.8,5.7,5.7,65.0,2.2,338.0,11.1,11.1,0.0,16.1,...,43.2,0.0,17.7,100.0,762.0,1022.0,8.9,2.8,5.1,0.0
2024-02-29 16:00:00,7.8,4.0,4.0,65.0,2.2,338.0,16.7,16.7,0.0,16.1,...,43.2,0.0,40.2,100.0,671.0,1017.4,8.9,2.8,5.0,0.0
2024-02-29 17:00:00,7.2,2.9,2.9,61.0,0.0,338.0,18.5,18.5,0.0,16.1,...,62.1,0.0,32.2,91.0,579.0,1017.7,8.3,0.6,6.2,0.0
2024-02-29 18:00:00,6.1,2.6,2.6,60.0,-1.1,338.0,14.8,37.0,0.0,16.1,...,67.0,0.0,24.1,13.0,12192.0,1024.0,7.8,1.1,3.9,0.0
2024-02-29 19:00:00,5.0,0.7,0.7,65.0,-1.1,338.0,16.7,16.7,0.0,16.1,...,67.0,0.0,49.9,13.0,10363.0,1018.7,7.2,-0.6,3.7,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-06 04:00:00,15.0,15.8,13.0,51,5.0,0,14.8,14.8,3,16.1,...,43.5,3,38.6,13,10211,1023.0,16.1,14.4,8.9,0.0
2024-11-06 05:00:00,15.0,16.3,13.7,51,5.0,338,11.1,11.1,3,16.1,...,27.0,3,45.1,0,10150,1022.5,17.2,14.4,10.7,0.0
2024-11-06 06:00:00,13.9,12.8,11.4,50,3.9,315,16.7,16.7,2,16.1,...,38.9,2,41.8,28,10180,1022.9,17.2,14.4,11.0,0.0


## 예측 가격 지역_기상

In [ ]:
df_1_predict

,location,base_ts,ts,temp,real_feel_temp,wet_bulb_temp,dew_point,wind_dir,wind_spd,wind_gust_spd,...,precip_prob,rain_prob,snow_prob,ice_prob,total_liq,rain,snow,ice,cld_cvr,new_ts
0,Ilgwa-ri,1709258400,1.709305e+09,3.33333,-3.88889,1.11111,-2.77778,343.0,27.8417,59.2239,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81.0,2024-03-01 15:00:00
1,Ilgwa-ri,1709258400,1.709309e+09,3.33333,-3.88889,1.11111,-2.77778,342.0,27.8417,59.2239,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88.0,2024-03-01 16:00:00
2,Ilgwa-ri,1709258400,1.709312e+09,3.33333,-4.44444,1.11111,-2.77778,342.0,27.8417,59.2239,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.0,2024-03-01 17:00:00
3,Ilgwa-ri,1709258400,1.709316e+09,2.77778,-4.44444,1.11111,-2.22222,341.0,29.6119,57.4536,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,94.0,2024-03-01 18:00:00
4,Ilgwa-ri,1709258400,1.709320e+09,3.33333,-4.44444,1.11111,-2.77778,341.0,29.6119,57.4536,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81.0,2024-03-01 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
56703,Sangmo-ri,1730858400,1.730977e+09,16.6667,16.1111,13.3333,10.5556,40,7.40298,18.5075,...,0,0,0,0,0.0,0.0,0,0,78,2024-11-07 11:00:00
56704,Sangmo-ri,1730858400,1.730981e+09,16.6667,16.1111,13.3333,11.1111,43,9.3342,22.2089,...,0,0,0,0,0.0,0.0,0,0,78,2024-11-07 12:00:00
56705,Sangmo-ri,1730858400,1.730984e+09,16.1111,15.5556,13.3333,11.1111,52,9.3342,22.2089,...,0,0,0,0,0.0,0.0,0,0,80,2024-11-07 13:00:00
56706,Sangmo-ri,1730858400,1.730988e+09,16.1111,15.5556,13.8889,11.6667,60,9.3342,22.2089,...,0,0,0,0,0.0,0.0,0,0,83,2024-11-07 14:00:00


In [ ]:
# 필요 없는 열 제거 및 new_ts 인덱스 설정
df_1_predict.drop(columns=['ts', 'base_ts'], inplace=True)
df_1_predict['new_ts'] = pd.to_datetime(df_1_predict['new_ts'])
df_1_predict.set_index("new_ts", inplace=True)

In [ ]:
df_1_predict

,location,temp,real_feel_temp,wet_bulb_temp,dew_point,wind_dir,wind_spd,wind_gust_spd,rel_hum,vis,...,uv_idx,precip_prob,rain_prob,snow_prob,ice_prob,total_liq,rain,snow,ice,cld_cvr
new_ts,,,,,,,,,,,,,,,,,,,,,
2024-03-01 15:00:00,Ilgwa-ri,3.33333,-3.88889,1.11111,-2.77778,343.0,27.8417,59.2239,64.0,16.0934,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81.0
2024-03-01 16:00:00,Ilgwa-ri,3.33333,-3.88889,1.11111,-2.77778,342.0,27.8417,59.2239,64.0,16.0934,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,88.0
2024-03-01 17:00:00,Ilgwa-ri,3.33333,-4.44444,1.11111,-2.77778,342.0,27.8417,59.2239,65.0,16.0934,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,91.0
2024-03-01 18:00:00,Ilgwa-ri,2.77778,-4.44444,1.11111,-2.22222,341.0,29.6119,57.4536,67.0,16.0934,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,94.0
2024-03-01 19:00:00,Ilgwa-ri,3.33333,-4.44444,1.11111,-2.77778,341.0,29.6119,57.4536,65.0,16.0934,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,81.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-07 11:00:00,Sangmo-ri,16.6667,16.1111,13.3333,10.5556,40,7.40298,18.5075,68,16.0934,...,0,0,0,0,0,0.0,0.0,0,0,78
2024-11-07 12:00:00,Sangmo-ri,16.6667,16.1111,13.3333,11.1111,43,9.3342,22.2089,69,16.0934,...,0,0,0,0,0,0.0,0.0,0,0,78
2024-11-07 13:00:00,Sangmo-ri,16.1111,15.5556,13.3333,11.1111,52,9.3342,22.2089,74,16.0934,...,0,0,0,0,0,0.0,0.0,0,0,80


In [ ]:

df_list = []
common_index = df_1_predict.index.unique()  # 공통 인덱스 정의

for location in df_1_predict['location'].unique():
    # 특정 location의 데이터만 선택
    temp_df = df_1_predict[df_1_predict['location'] == location].copy()

    # 중복된 인덱스 처리 (집계 함수로 해결, 여기서는 'first')
    temp_df = temp_df[~temp_df.index.duplicated(keep='first')]

    # 열 이름 변경 (location 정보 추가)
    temp_df.columns = [f"{location}_{col}_predict" if col != "location" else col for col in temp_df.columns]

    # location 열 삭제
    temp_df.drop(columns=["location"], inplace=True)

    # 공통 인덱스를 기준으로 재인덱싱
    temp_df = temp_df.reindex(common_index)

    # 결과를 리스트에 저장
    df_list.append(temp_df)

# location별 데이터프레임 병합
df_renamed_1_predict = pd.concat(df_list, axis=1)

# 결과 확인
df_renamed_1_predict

<ipython-input-12-c3a6df64a2d0>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  temp_df.drop(columns=["location"], inplace=True)


,Ilgwa-ri_temp_predict,Ilgwa-ri_real_feel_temp_predict,Ilgwa-ri_wet_bulb_temp_predict,Ilgwa-ri_dew_point_predict,Ilgwa-ri_wind_dir_predict,Ilgwa-ri_wind_spd_predict,Ilgwa-ri_wind_gust_spd_predict,Ilgwa-ri_rel_hum_predict,Ilgwa-ri_vis_predict,Ilgwa-ri_ceiling_predict,...,Sangmo-ri_uv_idx_predict,Sangmo-ri_precip_prob_predict,Sangmo-ri_rain_prob_predict,Sangmo-ri_snow_prob_predict,Sangmo-ri_ice_prob_predict,Sangmo-ri_total_liq_predict,Sangmo-ri_rain_predict,Sangmo-ri_snow_predict,Sangmo-ri_ice_predict,Sangmo-ri_cld_cvr_predict
new_ts,,,,,,,,,,,,,,,,,,,,,
2024-03-01 15:00:00,3.33333,-3.88889,1.11111,-2.77778,343.0,27.8417,59.2239,64.0,16.0934,609.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0
2024-03-01 16:00:00,3.33333,-3.88889,1.11111,-2.77778,342.0,27.8417,59.2239,64.0,16.0934,609.6,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86.0
2024-03-01 17:00:00,3.33333,-4.44444,1.11111,-2.77778,342.0,27.8417,59.2239,65.0,16.0934,1981.2,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0
2024-03-01 18:00:00,2.77778,-4.44444,1.11111,-2.22222,341.0,29.6119,57.4536,67.0,16.0934,3596.64,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,94.0
2024-03-01 19:00:00,3.33333,-4.44444,1.11111,-2.77778,341.0,29.6119,57.4536,65.0,16.0934,4998.72,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-07 11:00:00,16.6667,16.1111,13.3333,10.5556,47,7.40298,18.5075,68,16.0934,9144.0,...,0,0,0,0,0,0.0,0.0,0,0,78
2024-11-07 12:00:00,16.6667,16.1111,13.3333,11.1111,48,9.3342,22.2089,69,16.0934,9144.0,...,0,0,0,0,0,0.0,0.0,0,0,78
2024-11-07 13:00:00,16.1111,15.5556,13.3333,11.1111,52,9.3342,22.2089,74,16.0934,9144.0,...,0,0,0,0,0,0.0,0.0,0,0,80


In [ ]:
df_merged_1 = pd.merge(df_renamed_1_actual, df_renamed_1_predict, left_index=True, right_index=True)
df_merged_1

,Bonggae-dong_temp_actual,Bonggae-dong_real_feel_temp_actual,Bonggae-dong_real_feel_temp_shade_actual,Bonggae-dong_rel_hum_actual,Bonggae-dong_dew_point_actual,Bonggae-dong_wind_dir_actual,Bonggae-dong_wind_spd_actual,Bonggae-dong_wind_gust_spd_actual,Bonggae-dong_uv_idx_actual,Bonggae-dong_vis_actual,...,Sangmo-ri_uv_idx_predict,Sangmo-ri_precip_prob_predict,Sangmo-ri_rain_prob_predict,Sangmo-ri_snow_prob_predict,Sangmo-ri_ice_prob_predict,Sangmo-ri_total_liq_predict,Sangmo-ri_rain_predict,Sangmo-ri_snow_predict,Sangmo-ri_ice_predict,Sangmo-ri_cld_cvr_predict
2024-03-01 15:00:00,1.1,-5.7,-5.7,69.0,-3.9,338.0,18.5,18.5,0.0,16.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0
2024-03-01 16:00:00,2.2,-4.3,-4.3,55.0,-6.1,315.0,22.2,22.2,0.0,16.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,86.0
2024-03-01 17:00:00,2.2,-5.4,-5.4,64.0,-3.9,315.0,22.2,22.2,0.0,16.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,90.0
2024-03-01 18:00:00,2.2,-2.6,-2.6,64.0,-3.9,338.0,16.7,16.7,0.0,16.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,94.0
2024-03-01 19:00:00,1.1,-5.1,-5.1,69.0,-3.9,338.0,16.7,16.7,0.0,16.1,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,79.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-06 04:00:00,15.0,15.8,13.0,51,5.0,0,14.8,14.8,3,16.1,...,3,0,0,0,0,0.0,0.0,0,0,16
2024-11-06 05:00:00,15.0,16.3,13.7,51,5.0,338,11.1,11.1,3,16.1,...,3,0,0,0,0,0.0,0.0,0,0,8
2024-11-06 06:00:00,13.9,12.8,11.4,50,3.9,315,16.7,16.7,2,16.1,...,2,0,0,0,0,0.0,0.0,0,0,0
2024-11-06 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1,0,0,0,0,0.0,0.0,0,0,1


# 기상데이터_2 전처리

In [ ]:
df_2_actual = pd.read_csv('/content/drive/MyDrive/산업정보공학특론/사용데이터/통합기상실측데이터_2.csv')

In [ ]:
df_2_predict = pd.read_csv('/content/drive/MyDrive/산업정보공학특론/사용데이터/통합기상예측데이터_2.csv')

## 실측데이터

In [ ]:
df_2_actual

,location,ts,cloud,temp,temp_max,temp_min,humidity,ground_press,wind_speed,wind_dir,rain,snow,new_ts
0,Gaigeturi,1.709219e+09,80.0,8.03,8.03,8.03,73.0,1023.0,10.34,352.0,0.0,0.0,2024-02-29 15:00:56
1,Gaigeturi,1.709221e+09,75.0,8.03,8.03,8.03,66.0,1022.0,3.09,340.0,0.0,0.0,2024-02-29 15:31:04
2,Gaigeturi,1.709222e+09,85.0,7.75,7.75,7.75,71.0,1023.0,11.28,350.0,0.0,0.0,2024-02-29 16:00:54
3,Gaigeturi,1.709224e+09,75.0,8.03,8.03,8.03,66.0,1022.0,4.63,340.0,0.0,0.0,2024-02-29 16:31:16
4,Gaigeturi,1.709226e+09,88.0,7.11,7.11,7.11,66.0,1024.0,11.8,352.0,0.31,0.0,2024-02-29 17:00:58
...,...,...,...,...,...,...,...,...,...,...,...,...,...
36131,Jeju,1.730875e+09,8,13.75,13.75,13.75,56,1026,8.08,345,0.0,0,2024-11-06 06:30:38
36132,Jeju,1.730876e+09,8,13.75,13.75,13.75,56,1026,8.08,345,0.0,0,2024-11-06 07:00:51
36133,Jeju,1.730878e+09,40,14.96,14.96,14.96,47,1027,4.12,320,0.0,0,2024-11-06 07:30:41
36134,Jeju,1.730880e+09,7,13.25,13.25,13.25,58,1027,7.6,348,0.0,0,2024-11-06 08:01:16


In [ ]:
df_2_actual['new_ts'] = pd.to_datetime(df_2_actual['new_ts'])

# 2. datetime에서 시간(hour) 정보만 추출
df_2_actual['hour'] = df_2_actual['new_ts'].dt.floor('H')  # 시간 단위로 내림 처리
df_2_actual['time'] = df_2_actual['new_ts'].dt.floor('H')
# 3. 동일 location, hour에서 가장 초반 데이터 선택
df_filtered = (
    df_2_actual.sort_values(by=['location', 'hour', 'new_ts'])
    .groupby(['location', 'hour'])
    .first()
    .reset_index()
)

# 4. 결과 데이터프레임
df_result_2_actual = df_filtered.drop(columns=['hour','new_ts','ts'])  # 필요 없어진 열 제거

<ipython-input-17-9cad89449b35>:4: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_2_actual['hour'] = df_2_actual['new_ts'].dt.floor('H')  # 시간 단위로 내림 처리
<ipython-input-17-9cad89449b35>:5: FutureWarning: 'H' is deprecated and will be removed in a future version, please use 'h' instead.
  df_2_actual['time'] = df_2_actual['new_ts'].dt.floor('H')


In [ ]:
df_result_2_actual

,location,cloud,temp,temp_max,temp_min,humidity,ground_press,wind_speed,wind_dir,rain,snow,time
0,Cheju-do,95.0,3.23,3.23,3.23,75.0,1022.0,7.09,343.0,0.4,0.0,2024-02-29 15:00:00
1,Cheju-do,96.0,2.96,2.96,2.96,77.0,1023.0,8.01,342.0,0.0,0.0,2024-02-29 16:00:00
2,Cheju-do,97.0,2.35,2.35,2.35,72.0,1023.0,8.54,344.0,0.18,0.0,2024-02-29 17:00:00
3,Cheju-do,93.0,1.66,1.66,1.66,68.0,1024.0,8.51,343.0,0.0,0.0,2024-02-29 18:00:00
4,Cheju-do,47.0,1.27,1.27,1.27,70.0,1023.0,9.35,341.0,0.0,0.0,2024-02-29 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...
18049,Jeju,19,14.34,14.34,14.34,61,1026,7.86,355,0.0,0,2024-11-06 04:00:00
18050,Jeju,22,14.09,14.09,14.09,58,1026,6.67,346,0.0,0,2024-11-06 05:00:00
18051,Jeju,11,14.34,14.34,14.34,56,1026,8.39,345,0.0,0,2024-11-06 06:00:00
18052,Jeju,8,13.75,13.75,13.75,56,1026,8.08,345,0.0,0,2024-11-06 07:00:00


In [ ]:
df_result_2_actual.set_index("time", inplace=True)

# 2. 열 이름을 `지역_변수` 형식으로 변경
# location별로 데이터 처리
df_list = []
for location in df_result_2_actual['location'].unique():
    # 특정 location의 데이터만 선택
    temp_df = df_result_2_actual[df_result_2_actual['location'] == location].copy()
    # 열 이름 변경
    temp_df.columns = [f"{location}_{col}_actual" if col != "location" else col for col in temp_df.columns]
    # location 열 삭제
    temp_df.drop(columns=["location"], inplace=True)
    # 결과를 리스트에 저장
    df_list.append(temp_df)

# 3. location별 데이터프레임 병합
df_renamed_2_actual = pd.concat(df_list, axis=1)

In [ ]:
df_renamed_2_actual

,Cheju-do_cloud_actual,Cheju-do_temp_actual,Cheju-do_temp_max_actual,Cheju-do_temp_min_actual,Cheju-do_humidity_actual,Cheju-do_ground_press_actual,Cheju-do_wind_speed_actual,Cheju-do_wind_dir_actual,Cheju-do_rain_actual,Cheju-do_snow_actual,...,Jeju_cloud_actual,Jeju_temp_actual,Jeju_temp_max_actual,Jeju_temp_min_actual,Jeju_humidity_actual,Jeju_ground_press_actual,Jeju_wind_speed_actual,Jeju_wind_dir_actual,Jeju_rain_actual,Jeju_snow_actual
time,,,,,,,,,,,,,,,,,,,,,
2024-02-29 15:00:00,95.0,3.23,3.23,3.23,75.0,1022.0,7.09,343.0,0.4,0.0,...,93.0,7.18,7.18,7.18,76.0,1023.0,8.23,341.0,0.11,0.0
2024-02-29 16:00:00,96.0,2.96,2.96,2.96,77.0,1023.0,8.01,342.0,0.0,0.0,...,95.0,6.87,6.87,6.87,77.0,1023.0,8.98,340.0,0.0,0.0
2024-02-29 17:00:00,97.0,2.35,2.35,2.35,72.0,1023.0,8.54,344.0,0.18,0.0,...,96.0,6.15,6.15,6.15,71.0,1024.0,9.52,342.0,0.77,0.0
2024-02-29 18:00:00,93.0,1.66,1.66,1.66,68.0,1024.0,8.51,343.0,0.0,0.0,...,97.0,5.79,5.79,5.79,67.0,1024.0,9.4,340.0,0.0,0.0
2024-02-29 19:00:00,47.0,1.27,1.27,1.27,70.0,1023.0,9.35,341.0,0.0,0.0,...,53.0,5.19,5.19,5.19,69.0,1024.0,9.94,340.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-06 04:00:00,15,10.81,10.81,10.81,58,1025,7.02,355,0.0,0,...,19,14.34,14.34,14.34,61,1026,7.86,355,0.0,0
2024-11-06 05:00:00,18,10.46,10.46,10.46,56,1025,5.88,349,0.0,0,...,22,14.09,14.09,14.09,58,1026,6.67,346,0.0,0
2024-11-06 06:00:00,8,10.48,10.48,10.48,55,1025,7.25,346,0.0,0,...,11,14.34,14.34,14.34,56,1026,8.39,345,0.0,0


## 예측데이터

In [ ]:
df_2_predict

,location,base_ts,ts,cloud,temp,temp_max,temp_min,humidity,ground_press,wind_speed,wind_dir,rain,snow,new_ts
0,Gaigeturi,1709258400,1.709305e+09,100.0,3.23,3.23,3.23,59.0,1027.0,9.59,337.0,0.0,0.0,2024-03-01 15:00:00
1,Gaigeturi,1709258400,1.709309e+09,100.0,3.3,3.3,3.3,59.0,1027.0,9.28,338.0,0.0,0.0,2024-03-01 16:00:00
2,Gaigeturi,1709258400,1.709312e+09,100.0,3.35,3.35,3.35,59.0,1027.0,8.82,337.0,0.0,0.0,2024-03-01 17:00:00
3,Gaigeturi,1709258400,1.709316e+09,100.0,3.35,3.35,3.35,59.0,1026.0,8.42,338.0,0.0,0.0,2024-03-01 18:00:00
4,Gaigeturi,1709258400,1.709320e+09,99.0,3.43,3.43,3.43,59.0,1026.0,8.24,337.0,0.0,0.0,2024-03-01 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18897,Jeju,1730858400,1.730977e+09,3,13.58,13.58,13.58,77,1029,2.39,71,0.0,0,2024-11-07 11:00:00
18898,Jeju,1730858400,1.730981e+09,7,13.75,13.75,13.75,78,1029,2.61,61,0.0,0,2024-11-07 12:00:00
18899,Jeju,1730858400,1.730984e+09,40,14.03,14.03,14.03,79,1029,2.76,64,0.0,0,2024-11-07 13:00:00
18900,Jeju,1730858400,1.730988e+09,62,14.47,14.47,14.47,80,1029,3.05,59,0.0,0,2024-11-07 14:00:00


In [ ]:
# 필요 없는 열 제거 및 new_ts 인덱스 설정
df_2_predict.drop(columns=['ts', 'base_ts'], inplace=True)
df_2_predict['new_ts'] = pd.to_datetime(df_2_predict['new_ts'])
df_2_predict.set_index("new_ts", inplace=True)

In [ ]:
df_list = []
common_index = df_2_predict.index.unique()  # 공통 인덱스 정의

for location in df_2_predict['location'].unique():
    # 특정 location의 데이터만 선택
    temp_df = df_2_predict[df_2_predict['location'] == location].copy()

    # 중복된 인덱스 처리 (집계 함수로 해결, 여기서는 'first')
    temp_df = temp_df[~temp_df.index.duplicated(keep='first')]

    # 열 이름 변경 (location 정보 추가)
    temp_df.columns = [f"{location}_{col}_predict" if col != "location" else col for col in temp_df.columns]

    # location 열 삭제
    temp_df.drop(columns=["location"], inplace=True)

    # 공통 인덱스를 기준으로 재인덱싱
    temp_df = temp_df.reindex(common_index)

    # 결과를 리스트에 저장
    df_list.append(temp_df)

# location별 데이터프레임 병합
df_renamed_2_predict = pd.concat(df_list, axis=1)

# 결과 확인
df_renamed_2_predict

,Gaigeturi_cloud_predict,Gaigeturi_temp_predict,Gaigeturi_temp_max_predict,Gaigeturi_temp_min_predict,Gaigeturi_humidity_predict,Gaigeturi_ground_press_predict,Gaigeturi_wind_speed_predict,Gaigeturi_wind_dir_predict,Gaigeturi_rain_predict,Gaigeturi_snow_predict,...,Jeju_cloud_predict,Jeju_temp_predict,Jeju_temp_max_predict,Jeju_temp_min_predict,Jeju_humidity_predict,Jeju_ground_press_predict,Jeju_wind_speed_predict,Jeju_wind_dir_predict,Jeju_rain_predict,Jeju_snow_predict
new_ts,,,,,,,,,,,,,,,,,,,,,
2024-03-01 15:00:00,100.0,3.23,3.23,3.23,59.0,1027.0,9.59,337.0,0.0,0.0,...,100.0,2.23,2.23,2.23,61.0,1027.0,8.28,328.0,0.0,0.0
2024-03-01 16:00:00,100.0,3.3,3.3,3.3,59.0,1027.0,9.28,338.0,0.0,0.0,...,100.0,2.33,2.33,2.33,62.0,1027.0,8.07,330.0,0.0,0.0
2024-03-01 17:00:00,100.0,3.35,3.35,3.35,59.0,1027.0,8.82,337.0,0.0,0.0,...,100.0,2.39,2.39,2.39,61.0,1027.0,7.62,329.0,0.0,0.0
2024-03-01 18:00:00,100.0,3.35,3.35,3.35,59.0,1026.0,8.42,338.0,0.0,0.0,...,100.0,2.39,2.39,2.39,61.0,1026.0,7.33,328.0,0.0,0.0
2024-03-01 19:00:00,99.0,3.43,3.43,3.43,59.0,1026.0,8.24,337.0,0.0,0.0,...,100.0,2.4,2.4,2.4,61.0,1026.0,7.11,327.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-07 11:00:00,1,14.63,14.63,14.63,70,1029,3.64,56,0.0,0,...,3,13.58,13.58,13.58,77,1029,2.39,71,0.0,0
2024-11-07 12:00:00,4,14.8,14.8,14.8,72,1029,3.74,53,0.0,0,...,7,13.75,13.75,13.75,78,1029,2.61,61,0.0,0
2024-11-07 13:00:00,47,14.89,14.89,14.89,73,1029,3.86,52,0.0,0,...,40,14.03,14.03,14.03,79,1029,2.76,64,0.0,0


In [ ]:
df_merged_2 = pd.merge(df_renamed_2_actual, df_renamed_2_predict, left_index=True, right_index=True)
df_merged_2

,Cheju-do_cloud_actual,Cheju-do_temp_actual,Cheju-do_temp_max_actual,Cheju-do_temp_min_actual,Cheju-do_humidity_actual,Cheju-do_ground_press_actual,Cheju-do_wind_speed_actual,Cheju-do_wind_dir_actual,Cheju-do_rain_actual,Cheju-do_snow_actual,...,Jeju_cloud_predict,Jeju_temp_predict,Jeju_temp_max_predict,Jeju_temp_min_predict,Jeju_humidity_predict,Jeju_ground_press_predict,Jeju_wind_speed_predict,Jeju_wind_dir_predict,Jeju_rain_predict,Jeju_snow_predict
2024-03-01 15:00:00,100.0,-1.77,-1.77,-1.77,61.0,1027.0,7.46,332.0,0.0,0.0,...,100.0,2.23,2.23,2.23,61.0,1027.0,8.28,328.0,0.0,0.0
2024-03-01 16:00:00,100.0,-1.78,-1.78,-1.78,61.0,1027.0,7.18,331.0,0.0,0.0,...,100.0,2.33,2.33,2.33,62.0,1027.0,8.07,330.0,0.0,0.0
2024-03-01 17:00:00,100.0,-1.82,-1.82,-1.82,62.0,1026.0,7.14,330.0,0.0,0.0,...,100.0,2.39,2.39,2.39,61.0,1027.0,7.62,329.0,0.0,0.0
2024-03-01 18:00:00,99.0,-1.84,-1.84,-1.84,62.0,1026.0,6.41,331.0,0.0,0.0,...,100.0,2.39,2.39,2.39,61.0,1026.0,7.33,328.0,0.0,0.0
2024-03-01 19:00:00,100.0,-1.97,-1.97,-1.97,62.0,1025.0,6.46,331.0,0.0,0.0,...,100.0,2.4,2.4,2.4,61.0,1026.0,7.11,327.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-06 04:00:00,15,10.81,10.81,10.81,58,1025,7.02,355,0.0,0,...,46,15.1,15.1,15.1,54,1026,8.72,338,0.0,0
2024-11-06 05:00:00,18,10.46,10.46,10.46,56,1025,5.88,349,0.0,0,...,40,14.91,14.91,14.91,55,1025,8.56,342,0.0,0
2024-11-06 06:00:00,8,10.48,10.48,10.48,55,1025,7.25,346,0.0,0,...,35,14.45,14.45,14.45,56,1026,8.86,346,0.0,0
2024-11-06 07:00:00,6,9.99,9.99,9.99,55,1026,7.0,347,0.0,0,...,13,14.17,14.17,14.17,57,1026,8.49,347,0.0,0


# 최종 데이터 통합

In [ ]:
df_final = pd.merge(df_merged_1, df_merged_2, left_index=True, right_index=True)
df_final

,Bonggae-dong_temp_actual,Bonggae-dong_real_feel_temp_actual,Bonggae-dong_real_feel_temp_shade_actual,Bonggae-dong_rel_hum_actual,Bonggae-dong_dew_point_actual,Bonggae-dong_wind_dir_actual,Bonggae-dong_wind_spd_actual,Bonggae-dong_wind_gust_spd_actual,Bonggae-dong_uv_idx_actual,Bonggae-dong_vis_actual,...,Jeju_cloud_predict,Jeju_temp_predict,Jeju_temp_max_predict,Jeju_temp_min_predict,Jeju_humidity_predict,Jeju_ground_press_predict,Jeju_wind_speed_predict,Jeju_wind_dir_predict,Jeju_rain_predict,Jeju_snow_predict
2024-03-01 15:00:00,1.1,-5.7,-5.7,69.0,-3.9,338.0,18.5,18.5,0.0,16.1,...,100.0,2.23,2.23,2.23,61.0,1027.0,8.28,328.0,0.0,0.0
2024-03-01 16:00:00,2.2,-4.3,-4.3,55.0,-6.1,315.0,22.2,22.2,0.0,16.1,...,100.0,2.33,2.33,2.33,62.0,1027.0,8.07,330.0,0.0,0.0
2024-03-01 17:00:00,2.2,-5.4,-5.4,64.0,-3.9,315.0,22.2,22.2,0.0,16.1,...,100.0,2.39,2.39,2.39,61.0,1027.0,7.62,329.0,0.0,0.0
2024-03-01 18:00:00,2.2,-2.6,-2.6,64.0,-3.9,338.0,16.7,16.7,0.0,16.1,...,100.0,2.39,2.39,2.39,61.0,1026.0,7.33,328.0,0.0,0.0
2024-03-01 19:00:00,1.1,-5.1,-5.1,69.0,-3.9,338.0,16.7,16.7,0.0,16.1,...,100.0,2.4,2.4,2.4,61.0,1026.0,7.11,327.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-06 04:00:00,15.0,15.8,13.0,51,5.0,0,14.8,14.8,3,16.1,...,46,15.1,15.1,15.1,54,1026,8.72,338,0.0,0
2024-11-06 05:00:00,15.0,16.3,13.7,51,5.0,338,11.1,11.1,3,16.1,...,40,14.91,14.91,14.91,55,1025,8.56,342,0.0,0
2024-11-06 06:00:00,13.9,12.8,11.4,50,3.9,315,16.7,16.7,2,16.1,...,35,14.45,14.45,14.45,56,1026,8.86,346,0.0,0
2024-11-06 07:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,13,14.17,14.17,14.17,57,1026,8.49,347,0.0,0


In [ ]:
df_price = pd.read_csv('/content/drive/MyDrive/산업정보공학특론/사용데이터/최종데이터/기상1_가격_통합데이터.csv')

In [ ]:
df_price.columns

Index(['location', 'hour', 'temp_actual', 'real_feel_temp_actual',
       'real_feel_temp_shade', 'rel_hum_actual', 'dew_point_actual',
       'wind_dir_actual', 'wind_spd_actual', 'wind_gust_spd_actual',
       'uv_idx_actual', 'vis_actual', 'cld_cvr_actual', 'ceiling_actual',
       'pressure', 'appr_temp', 'wind_chill_temp', 'wet_bulb_temp_actual',
       'precip_1h', 'temp_predict', 'real_feel_temp_predict',
       'wet_bulb_temp_predict', 'dew_point_predict', 'wind_dir_predict',
       'wind_spd_predict', 'wind_gust_spd_predict', 'rel_hum_predict',
       'vis_predict', 'ceiling_predict', 'uv_idx_predict', 'precip_prob',
       'rain_prob', 'snow_prob', 'ice_prob', 'total_liq', 'rain', 'snow',
       'ice', 'cld_cvr_predict', 'smp_rt', 'smp_rc', 'smp_da', 'supply_power',
       'present_load', 'power_solar', 'power_wind', 'renewable_energy_total',
       'supply_capacity', 'operation_capacity'],
      dtype='object')

In [ ]:
df_price = df_price[['hour','smp_rt', 'smp_rc', 'smp_da', 'supply_power',
       'present_load', 'power_solar', 'power_wind', 'renewable_energy_total',
       'supply_capacity', 'operation_capacity']]


In [ ]:
# 'hour' 기준으로 중복 제거
df_price["hour"] = pd.to_datetime(df_price["hour"])  # Ensure 'hour' is in datetime format
df_unique = df_price.drop_duplicates(subset=["hour"], keep="first")
df_unique.set_index("hour", inplace=True)

In [ ]:
df_final = pd.merge(df_final, df_unique, left_index=True, right_index=True)
df_final

,Bonggae-dong_temp_actual,Bonggae-dong_real_feel_temp_actual,Bonggae-dong_real_feel_temp_shade_actual,Bonggae-dong_rel_hum_actual,Bonggae-dong_dew_point_actual,Bonggae-dong_wind_dir_actual,Bonggae-dong_wind_spd_actual,Bonggae-dong_wind_gust_spd_actual,Bonggae-dong_uv_idx_actual,Bonggae-dong_vis_actual,...,smp_rt,smp_rc,smp_da,supply_power,present_load,power_solar,power_wind,renewable_energy_total,supply_capacity,operation_capacity
2024-03-01 15:00:00,1.1,-5.7,-5.7,69.0,-3.9,338.0,18.5,18.5,0.0,16.1,...,154.8625,154.8075,112.19,1.343583e+06,869166.666667,0.000000,186993.583333,202769.000000,474583.333333,373750.000000
2024-03-01 16:00:00,2.2,-4.3,-4.3,55.0,-6.1,315.0,22.2,22.2,0.0,16.1,...,173.1925,147.0725,138.50,1.337417e+06,831166.666667,0.000000,181352.416667,197162.666667,506250.000000,395166.666667
2024-03-01 17:00:00,2.2,-5.4,-5.4,64.0,-3.9,315.0,22.2,22.2,0.0,16.1,...,167.9850,137.4925,99.48,1.318833e+06,807500.000000,0.000000,161981.666667,177954.083333,511250.000000,285833.333333
2024-03-01 18:00:00,2.2,-2.6,-2.6,64.0,-3.9,338.0,16.7,16.7,0.0,16.1,...,147.9225,126.0225,95.38,1.298750e+06,789833.333333,0.000000,140727.083333,156318.166667,508666.666667,283166.666667
2024-03-01 19:00:00,1.1,-5.1,-5.1,69.0,-3.9,338.0,16.7,16.7,0.0,16.1,...,149.7575,132.3200,91.24,1.285500e+06,785083.333333,0.000000,128531.500000,143984.916667,500416.666667,275833.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-05 11:00:00,13.9,11.4,11.4,58,6.1,338,16.7,16.7,0,16.1,...,127.6050,127.6050,127.56,1.237778e+06,686777.777778,5900.230000,298084.444444,304228.555556,551000.000000,375777.777778
2024-11-05 12:00:00,13.9,10.3,10.3,50,3.9,315,24.1,24.1,0,16.1,...,112.5000,112.5000,127.62,1.240083e+06,661250.000000,3233.595833,302066.500000,305544.083333,579083.333333,399333.333333
2024-11-05 13:00:00,13.9,10.8,10.8,54,5.0,315,20.4,20.4,0,16.1,...,105.7200,105.7200,107.46,1.240000e+06,638583.333333,615.570000,304275.416667,305420.500000,601416.666667,414500.000000
2024-11-05 14:00:00,13.9,11.0,11.0,50,3.9,338,18.5,18.5,0,16.1,...,104.9975,104.9975,105.14,1.212273e+06,612545.454545,0.000000,277198.727273,277602.727273,599545.454545,411545.454545


In [ ]:
# 1. 1시간 결측치는 바로 이전 값으로 채움
df_filled = df_final.ffill(limit=1)

# 2. 하루치 결측치 처리: 전날 같은 시간의 데이터로 채움
def fill_with_previous_day_vectorized(dataframe):
    # 결측치 마스크 생성
    null_mask = dataframe.isnull()

    # 전날 데이터 가져오기
    previous_day_index = dataframe.index - pd.Timedelta(days=1)
    valid_previous_day_mask = previous_day_index.isin(dataframe.index)

    # DataFrame을 직접 참조
    previous_day_data = dataframe.loc[previous_day_index[valid_previous_day_mask]]
    previous_day_data.index = dataframe.index[valid_previous_day_mask]

    # 결측치에 전날 데이터를 병합
    dataframe = dataframe.combine_first(previous_day_data)

    return dataframe

# 적용
df_filled = fill_with_previous_day_vectorized(df_filled)

df_filled

<ipython-input-31-3b10291d4869>:2: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_filled = df_final.ffill(limit=1)


,Bonggae-dong_temp_actual,Bonggae-dong_real_feel_temp_actual,Bonggae-dong_real_feel_temp_shade_actual,Bonggae-dong_rel_hum_actual,Bonggae-dong_dew_point_actual,Bonggae-dong_wind_dir_actual,Bonggae-dong_wind_spd_actual,Bonggae-dong_wind_gust_spd_actual,Bonggae-dong_uv_idx_actual,Bonggae-dong_vis_actual,...,smp_rt,smp_rc,smp_da,supply_power,present_load,power_solar,power_wind,renewable_energy_total,supply_capacity,operation_capacity
2024-03-01 15:00:00,1.1,-5.7,-5.7,69.0,-3.9,338.0,18.5,18.5,0.0,16.1,...,154.8625,154.8075,112.19,1.343583e+06,869166.666667,0.000000,186993.583333,202769.000000,474583.333333,373750.000000
2024-03-01 16:00:00,2.2,-4.3,-4.3,55.0,-6.1,315.0,22.2,22.2,0.0,16.1,...,173.1925,147.0725,138.50,1.337417e+06,831166.666667,0.000000,181352.416667,197162.666667,506250.000000,395166.666667
2024-03-01 17:00:00,2.2,-5.4,-5.4,64.0,-3.9,315.0,22.2,22.2,0.0,16.1,...,167.9850,137.4925,99.48,1.318833e+06,807500.000000,0.000000,161981.666667,177954.083333,511250.000000,285833.333333
2024-03-01 18:00:00,2.2,-2.6,-2.6,64.0,-3.9,338.0,16.7,16.7,0.0,16.1,...,147.9225,126.0225,95.38,1.298750e+06,789833.333333,0.000000,140727.083333,156318.166667,508666.666667,283166.666667
2024-03-01 19:00:00,1.1,-5.1,-5.1,69.0,-3.9,338.0,16.7,16.7,0.0,16.1,...,149.7575,132.3200,91.24,1.285500e+06,785083.333333,0.000000,128531.500000,143984.916667,500416.666667,275833.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-05 11:00:00,13.9,11.4,11.4,58,6.1,338,16.7,16.7,0,16.1,...,127.6050,127.6050,127.56,1.237778e+06,686777.777778,5900.230000,298084.444444,304228.555556,551000.000000,375777.777778
2024-11-05 12:00:00,13.9,10.3,10.3,50,3.9,315,24.1,24.1,0,16.1,...,112.5000,112.5000,127.62,1.240083e+06,661250.000000,3233.595833,302066.500000,305544.083333,579083.333333,399333.333333
2024-11-05 13:00:00,13.9,10.8,10.8,54,5.0,315,20.4,20.4,0,16.1,...,105.7200,105.7200,107.46,1.240000e+06,638583.333333,615.570000,304275.416667,305420.500000,601416.666667,414500.000000
2024-11-05 14:00:00,13.9,11.0,11.0,50,3.9,338,18.5,18.5,0,16.1,...,104.9975,104.9975,105.14,1.212273e+06,612545.454545,0.000000,277198.727273,277602.727273,599545.454545,411545.454545


In [ ]:
df_filled.dropna(axis = 1,inplace = True)

In [ ]:
# 결측치가 있는 열만 필터링
missing_values = df_filled.isnull().sum()  # 결측치 개수 계산
missing_values = missing_values[missing_values >0]  # 0보다 큰 값(결측치가 있는 열)만 필터링

# 결과 출력
print(missing_values)

Series([], dtype: int64)


In [ ]:
df_filled.to_csv('/content/drive/MyDrive/산업정보공학특론/사용데이터/중간이후_최종_241129.csv')

In [ ]:
df_filled

,Bonggae-dong_temp_actual,Bonggae-dong_real_feel_temp_actual,Bonggae-dong_real_feel_temp_shade_actual,Bonggae-dong_rel_hum_actual,Bonggae-dong_dew_point_actual,Bonggae-dong_wind_dir_actual,Bonggae-dong_wind_spd_actual,Bonggae-dong_wind_gust_spd_actual,Bonggae-dong_uv_idx_actual,Bonggae-dong_vis_actual,...,smp_rt,smp_rc,smp_da,supply_power,present_load,power_solar,power_wind,renewable_energy_total,supply_capacity,operation_capacity
2024-03-01 15:00:00,1.1,-5.7,-5.7,69.0,-3.9,338.0,18.5,18.5,0.0,16.1,...,154.8625,154.8075,112.19,1.343583e+06,869166.666667,0.000000,186993.583333,202769.000000,474583.333333,373750.000000
2024-03-01 16:00:00,2.2,-4.3,-4.3,55.0,-6.1,315.0,22.2,22.2,0.0,16.1,...,173.1925,147.0725,138.50,1.337417e+06,831166.666667,0.000000,181352.416667,197162.666667,506250.000000,395166.666667
2024-03-01 17:00:00,2.2,-5.4,-5.4,64.0,-3.9,315.0,22.2,22.2,0.0,16.1,...,167.9850,137.4925,99.48,1.318833e+06,807500.000000,0.000000,161981.666667,177954.083333,511250.000000,285833.333333
2024-03-01 18:00:00,2.2,-2.6,-2.6,64.0,-3.9,338.0,16.7,16.7,0.0,16.1,...,147.9225,126.0225,95.38,1.298750e+06,789833.333333,0.000000,140727.083333,156318.166667,508666.666667,283166.666667
2024-03-01 19:00:00,1.1,-5.1,-5.1,69.0,-3.9,338.0,16.7,16.7,0.0,16.1,...,149.7575,132.3200,91.24,1.285500e+06,785083.333333,0.000000,128531.500000,143984.916667,500416.666667,275833.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11-05 11:00:00,13.9,11.4,11.4,58,6.1,338,16.7,16.7,0,16.1,...,127.6050,127.6050,127.56,1.237778e+06,686777.777778,5900.230000,298084.444444,304228.555556,551000.000000,375777.777778
2024-11-05 12:00:00,13.9,10.3,10.3,50,3.9,315,24.1,24.1,0,16.1,...,112.5000,112.5000,127.62,1.240083e+06,661250.000000,3233.595833,302066.500000,305544.083333,579083.333333,399333.333333
2024-11-05 13:00:00,13.9,10.8,10.8,54,5.0,315,20.4,20.4,0,16.1,...,105.7200,105.7200,107.46,1.240000e+06,638583.333333,615.570000,304275.416667,305420.500000,601416.666667,414500.000000
2024-11-05 14:00:00,13.9,11.0,11.0,50,3.9,338,18.5,18.5,0,16.1,...,104.9975,104.9975,105.14,1.212273e+06,612545.454545,0.000000,277198.727273,277602.727273,599545.454545,411545.454545


In [ ]:
correlation_matrix = df_filled.corr()  # 모든 변수 간의 상관관계 계산
smp_rc_correlation = correlation_matrix["smp_rc"].sort_values(ascending=False)  # smp_rc와 다른 변수 간 상관관계 추출

# 2. 데이터프레임으로 저장
correlation_df = smp_rc_correlation.to_frame(name="correlation_with_smp_rc")

# 결과 출력
print(correlation_df)

                            correlation_with_smp_rc
smp_rc                                     1.000000
smp_rt                                     0.994809
smp_da                                     0.638180
present_load                               0.514909
Sangmo-ri_appr_temp_actual                 0.251861
...                                             ...
Gaigeturi_snow_actual                           NaN
Jeju_snow_actual                                NaN
Gaigeturi_snow_predict                          NaN
Cheju-do_snow_predict                           NaN
Jeju_snow_predict                               NaN

[403 rows x 1 columns]


In [ ]:
correlation_df_higher_20 = correlation_df[correlation_df["correlation_with_smp_rc"].abs() > 0.2]
correlation_df_higher_20

,correlation_with_smp_rc
smp_rc,1.000000
smp_rt,0.994809
smp_da,0.638180
present_load,0.514909
Sangmo-ri_appr_temp_actual,0.251861
...,...
Cheonji-dong_uv_idx_predict,-0.241737
Geumak-ri_uv_idx_predict,-0.245156
renewable_energy_total,-0.268589
operation_capacity,-0.363274


In [ ]:
correlation_df_higher_20.index

Index(['smp_rc', 'smp_rt', 'smp_da', 'present_load',
       'Sangmo-ri_appr_temp_actual', 'Songdang-ri_rel_hum_actual',
       'Ilgwa-ri_appr_temp_actual', 'Bonggae-dong_appr_temp_actual',
       'Hacheon-ri_rel_hum_actual', 'Gwangryeong-ri_dew_point_actual',
       'Bonggae-dong_dew_point_actual', 'Songdang-ri_dew_point_actual',
       'Yongsu-ri_appr_temp_actual', 'Hacheon-ri_dew_point_predict',
       'Songdang-ri_dew_point_predict',
       'Sangmo-ri_real_feel_temp_shade_actual',
       'Ilgwa-ri_real_feel_temp_shade_actual', 'Geumak-ri_dew_point_predict',
       'Hacheon-ri_dew_point_actual', 'Cheonji-dong_dew_point_predict',
       'Ilgwa-ri_dew_point_actual', 'Gwangryeong-ri_dew_point_predict',
       'Bonggae-dong_dew_point_predict', 'Gwangryeong-ri_appr_temp_actual',
       'Sangmo-ri_dew_point_predict', 'Sangmo-ri_dew_point_actual',
       'Ilgwa-ri_dew_point_predict', 'Cheonji-dong_appr_temp_actual',
       'Yongsu-ri_dew_point_actual', 'Cheonji-dong_dew_point_actual',
     

In [ ]:
correlation_df_higher_20.to_excel('aa.xlsx',index =True)